In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=f8f5c42cdca8c9c7fb5e390d464671f127aaf2bd3df6d7345930048a69c7b635
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
# Check spark session
print(spark)

In [4]:
spark.sparkContext.setLogLevel("ERROR")

In [5]:
from pyspark.sql.types import *
df_schema = StructType([StructField("link", StringType(), True)\
                       ,StructField("headline", StringType(), True)\
                       ,StructField("category", StringType(), True)\
                       ,StructField("short_description", StringType(), True)\
                       ,StructField("authors", StringType(), True)\
                       ,StructField("date", StringType(), True)])

In [12]:
df = spark.readStream.schema(df_schema).json("/content/input-05111940000202/")

In [13]:
groupDF = df.select("category").groupBy("category").count()

groupDF.printSchema()

root
 |-- category: string (nullable = true)
 |-- count: long (nullable = false)



In [14]:
result = groupDF.writeStream\
  .format("console")\
  .outputMode("complete")\
  .start()


result.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored